In [48]:
import datetime as dt
from datetime import date
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import time


def ticker_df(ticker, start, end):
    source = 'yahoo'
    df = web.DataReader(ticker, source, start, end)
    return df.drop(['High', 'Low', 'Open', 'Adj Close', 'Volume'], axis = 1)

def stock_correlation(stock1, stock2):
    correlation = stock1['Close'].pct_change().corr(stock2['Close'].pct_change())
    return(correlation)

def get_correlation(stocks, start, end):
    
    assets ={}
    assets2 = {}

    for stock in stocks:
        assets[stock] = ticker_df(stock, start, end)

    for stock in stocks:
        assets2[stock] = list(assets[stock]['Close'])

    df = pd.DataFrame(data = assets2, columns = assets.keys())
    corr_matrix = df.corr()

    for stock1 in stocks:
        for stock2 in stocks:
            corr_matrix[stock1][stock2] = stock_correlation(assets[stock1], assets[stock2])
    
    return(corr_matrix)



#takes holdings {'ticker':allocation}, returns simulation of portfolio
def sim_port(holdings, start, end):
    portfolio = ticker_df('SPY', start, end).drop('Close', axis =1)

    for stock in holdings.keys():
        portfolio[stock + ' close'] = ticker_df(stock, start, end)
        portfolio[stock + ' pct change'] = portfolio[stock + ' close'].pct_change()

    for stock in holdings.keys():

        portfolio[stock + ' sim close'] = [0.0]*len(portfolio)
        portfolio[stock + ' sim close'][0] = holdings[stock]

        for i in range(1, len(portfolio)):
            a = portfolio[stock + ' sim close'][i-1]
            b = 1 + portfolio[stock + ' pct change'][i]
            portfolio[stock + ' sim close'][i] =  a*b

    portfolio['Close'] = [0.0]*len(portfolio)

    for stock in holdings.keys():
        for i in range(len(portfolio)):
            portfolio['Close'][i] += portfolio[stock + ' sim close'][i]

    return(portfolio)

#takes list of stocks (strings), returns dictionary of time series
def asset_dict(stocks, start, end):
    assets = {}
    for stock in stocks:
        assets[stock] = ticker_df(stock,start,end)['Close']
    return(assets)

#takes list of stocks, returns dictionary of time series
def asset_timeseries(stocks, start, end):
    assets = {}
    for stock in stocks:
        assets[stock] = list(ticker_df(stock, start, end)['Close'])
    return(assets)


def series_corr(ser1, ser2):
    correlation = ser1.pct_change().corr(ser2.pct_change())
    return(correlation)

def create_corr_matrix(stock_dict):
    data = pd.DataFrame(data = stock_dict, columns = stock_dict.keys())
    corr_matrix = data.corr()

    for stock1 in stock_dict.keys():
        for stock2 in stock_dict.keys():
            corr_matrix[stock1][stock2] = series_corr(stock_dict[stock1], stock_dict[stock2])
    return(corr_matrix)


In [72]:
start = dt.datetime(2015,1,1)
end = dt.datetime(2020,6,30)

holdings= {'XSU.TO':30.0, 'XQQ.TO':25.0, 'VEF.TO':30.0, 'GDX': 5.0, 'ARKK':10.0,}
stocks = ['SPY', 'IWM', 'QQQ', 'GDX', 'GDXJ', 'GLD', 'SLV', 'USO', 'XLK', 'XLE', 'XRT', 'XRT', 'XLU', 'XLF','XLI','XLV', 'XAR', 'ARKK', 'XSU.TO']
assets = asset_dict(stocks, start, end)

portfolio = sim_port(holdings, start, end)
assets['portfolio'] = portfolio['Close']
create_corr_matrix(assets)

,SPY,IWM,QQQ,GDX,GDXJ,GLD,SLV,USO,XLK,XLE,XRT,XLU,XLF,XLI,XLV,XAR,ARKK,XSU.TO,portfolio
SPY,1.000000,0.901847,0.935922,0.081083,0.140907,-0.056771,0.165960,0.378633,0.943215,0.773610,0.784478,0.625923,0.902025,0.926928,0.878083,0.848603,0.761728,0.880655,0.928252
IWM,0.901847,1.000000,0.817599,0.084537,0.138616,-0.057306,0.179606,0.357117,0.817076,0.767254,0.850378,0.510905,0.872527,0.881299,0.772601,0.868747,0.792791,0.972041,0.912380
QQQ,0.935922,0.817599,1.000000,0.051894,0.110603,-0.054475,0.144819,0.308289,0.977861,0.631612,0.696720,0.506058,0.760708,0.801708,0.829209,0.734721,0.797392,0.800377,0.911626
GDX,0.081083,0.084537,0.051894,1.000000,0.951425,0.755357,0.633158,0.148195,0.050723,0.171530,0.039304,0.207333,-0.006613,0.080245,0.056321,0.110923,0.074283,0.096622,0.184372
GDXJ,0.140907,0.138616,0.110603,0.951425,1.000000,0.737030,0.627762,0.149633,0.107460,0.205583,0.092687,0.250332,0.056991,0.134201,0.106086,0.154597,0.121440,0.153691,0.240766
GLD,-0.056771,-0.057306,-0.054475,0.755357,0.737030,1.000000,0.766109,0.007532,-0.047268,-0.015975,-0.083399,0.166693,-0.151811,-0.071835,-0.040891,-0.025383,-0.024167,-0.030854,0.024542
SLV,0.165960,0.179606,0.144819,0.633158,0.627762,0.766109,1.000000,0.190252,0.149658,0.211815,0.121558,0.202900,0.089141,0.161786,0.131524,0.187696,0.184817,0.197506,0.250682
USO,0.378633,0.357117,0.308289,0.148195,0.149633,0.007532,0.190252,1.000000,0.321061,0.626340,0.280387,0.116741,0.346334,0.363764,0.251513,0.342842,0.267796,0.341340,0.362046
XLK,0.943215,0.817076,0.977861,0.050723,0.107460,-0.047268,0.149658,0.321061,1.000000,0.655206,0.692238,0.534804,0.784741,0.824845,0.805184,0.755098,0.778624,0.798728,0.902740
XLE,0.773610,0.767254,0.631612,0.171530,0.205583,-0.015975,0.211815,0.626340,0.655206,1.000000,0.648199,0.441470,0.761526,0.779056,0.606881,0.721107,0.575273,0.743866,0.728521
